In [1]:
# Some first fun with the scripts
# https://github.com/nlintz/TensorFlow-Tutorials/blob/master/08_word2vec.ipynb
# Inspired by https://www.tensorflow.org/versions/r0.7/tutorials/word2vec/index.html
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
# Read the data into a list of strings.
def read_data(filename):
  """Extract the files enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = []
    for name in f.namelist():
        print(name)
        print(type(f.read(name)))
        data.extend(tf.compat.as_str(f.read(name)).split())
        
    print(type(data))
  return data

In [34]:
# Configuration
batch_size = 20
# Dimension of the embedding vector. Two too small to get
# any meaningful embeddings, but let's make it 2 for simple visualization
embedding_size = 2
num_sampled = 15    # Number of negative examples to sample.

filename = '../scripts.zip'

"""
# Sample sentences
sentences = ["the quick brown fox jumped over the lazy dog",
            "I love cats and dogs",
            "we all love cats and dogs",
            "cats and dogs are great",
            "sung likes cats",
            "she loves dogs",
            "cats can be very independent",
            "cats are great companions when they want to be",
            "cats are playful",
            "cats are natural hunters",
            "It's raining cats and dogs",
            "dogs and cats love sung"]
"""

# sentences to words and count
#words = " ".join(sentences).split()
words = read_data(filename)
print('Data size', len(words))

# make all words lower case 

words = [word.lower() for word in words]

count = collections.Counter(words).most_common()
print(type(count))
print ("Word count", count[:50])

# Build dictionaries
rdic = [i[0] for i in count] #reverse dic, idx -> word
dic = {w: i for i, w in enumerate(rdic)} #dic, word -> id
voc_size = len(dic)

# Make indexed word data
data = [dic[word] for word in words]
print('Sample data', data[:10], [rdic[t] for t in data[:10]])

# Let's make a training data for window size 1 for simplicity
# ([the, brown], quick), ([quick, fox], brown), ([brown, jumped], fox), ...
cbow_pairs = [];
for i in range(1, len(data)-1) :
    cbow_pairs.append([[data[i-1], data[i+1]], data[i]]);
print('Context pairs', cbow_pairs[:10])

# Let's make skip-gram pairs
# (quick, the), (quick, brown), (brown, quick), (brown, fox), ...
skip_gram_pairs = [];
for c in cbow_pairs:
    skip_gram_pairs.append([c[1], c[0][0]])
    skip_gram_pairs.append([c[1], c[0][1]])
print('skip-gram pairs', skip_gram_pairs[:5])

def generate_batch(size):
    assert size < len(skip_gram_pairs)
    x_data=[]
    y_data = []
    r = np.random.choice(range(len(skip_gram_pairs)), size, replace=False)
    for i in r:
        x_data.append(skip_gram_pairs[i][0])  # n dim
        y_data.append([skip_gram_pairs[i][1]])  # n, 1 dim
    return x_data, y_data

# generate_batch test
print ('Batches (x, y)', generate_batch(3))

MOVIE_10 THINGS I HATE ABOUT YOU.txt
<class 'bytes'>
MOVIE_12 MONKEYS.txt
<class 'bytes'>
MOVIE_12 YEARS A SLAVE.txt
<class 'bytes'>
MOVIE_1492 CONQUEST OF PARADISE.txt
<class 'bytes'>
MOVIE_17 AGAIN.txt
<class 'bytes'>


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 342: invalid start byte